In [1]:
print("Loading Libraries...")
import os, sys
import csv
import platform
import numpy as np
import pandas as pd
import dask.dataframe as dd
import scipy
import sklearn
import matplotlib.pyplot as plt
import flask
import math
import scipy.interpolate as scin
import matplotlib.colors as colors
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm
from matplotlib.colors import LogNorm
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
from sklearn import neighbors, datasets
print("Loaded Libraries...")

Loading Libraries...
Loaded Libraries...


In [ ]:
print("Starting code...")

In [2]:
print("Loading directories..")
path_data = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-12'
path_post = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-12/post'
path_acu = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-12/post/acu'
path_plots = 'D:/01_Dokumenty/01_PUT/01_DOKTORAT/13_PLGRID/noise-data/int-12/post/plots' #ścieżka do katalogu z interesującymi nas plikami
print("Loaded directories...")

Loading directories..
Loaded directories...


In [ ]:
print("Loading batch data...")
os.chdir(path_data)
batch_data = dd.read_csv('int-01-*.dat', delimiter=r"\s+", decimal='.')
print("Batch data done...")

print("Calculating batch averages...")
averages = pd.DataFrame(batch_data.groupby('nodenumber').mean().compute())
print("Batch averages done...")

print("Generating node coordinates...")
avg_static_p = pd.DataFrame({'pressure': averages['pressure']})
node_coords = pd.DataFrame({
    'x-coordinate': averages['x-coordinate'],
    'y-coordinate': averages['y-coordinate'],
    'z-coordinate': averages['z-coordinate']
})
print("Node coords done...")

del(batch_data)
print("Batch data deleted...")

print("Listing files...")
filelist = os.listdir(path_data)

print("Starting noise analysis loop...")
for file in filelist:
    os.chdir(path_data)
    timestep = str(os.path.basename(str(file)))[7:-4]
    #time_static_p = pd.DataFrame(pd.read_csv(file, sep='/s+', header=0, usecols=["nodenumber", "pressure"], skiprows=0, decimal='.')).set_index('nodenumber')
    time_static_p = pd.DataFrame(pd.read_csv(file, delimiter=r"\s+", header=0, usecols=["nodenumber", "pressure"], skiprows=0, decimal='.')).set_index('nodenumber')
    acoustic_p = time_static_p.subtract(avg_static_p, fill_value=None)
    db = acoustic_p.apply(lambda x: 20 * np.log10(np.abs(x)/0.00002), axis=1)
    acoustic_data = pd.concat([node_coords, acoustic_p, db], axis=1)
    acoustic_data.columns = ['x-coordinate', 'y-coordinate', 'z-coordinate', 'sound-pressure', 'db-level']
    os.chdir(path_acu)
    acoustic_data.to_csv(str('int-01_acu_' + str(timestep) + '.dat'), sep=',')
    print(str('int-01_acu_' + str(timestep) + '.dat done...'))
print("Exiting noise analysis loop...")

In [4]:
print("Loading batch acoustic data...")
os.chdir(path_acu)
batch_data = dd.read_csv('int-12*.dat', sep=',', decimal='.')
print("Batch data done...")

print("Calculate min, max, std for plotting range...")
minima = pd.DataFrame(batch_data.groupby('nodenumber').min().compute())
maxima = pd.DataFrame(batch_data.groupby('nodenumber').max().compute())
min_spl=np.amin(minima['sound-pressure'])
min_dbl=np.amin(minima['db-level'])
max_spl=np.amax(maxima['sound-pressure'])
max_dbl=np.amax(maxima['db-level'])
std_devs = pd.DataFrame(batch_data.groupby('nodenumber').max().compute())
min_splrms=np.amin(std_devs['sound-pressure'])
min_dblrms=np.amin(std_devs['db-level'])
max_splrms=np.amax(std_devs['sound-pressure'])
max_dblrms=np.amax(std_devs['db-level'])
print("Min, max, std for plotting range done...")

print("Prepare to plot RMS values...")
x = std_devs['x-coordinate']
y = std_devs['y-coordinate']
z = std_devs['z-coordinate']
spl = std_devs['sound-pressure']
dbl = std_devs['db-level']
print("RMS values prep done...")

Loading batch acoustic data...
Batch data done...
Calculate min, max, std for plotting range...
Min, max, std for plotting range done...
Prepare to plot RMS values...
RMS values prep done...


In [12]:
print(min_splrms)
print(max_splrms)
print(min_dblrms)
print(max_dblrms)

-69425.266702
41775.389434
131.692438645
191.227876611


In [11]:
print("Plotting RMS values...")
fig, (ax0, ax1) = plt.subplots(nrows=2, figsize=(10, 10), dpi=300)
spl_plot = ax0.scatter(z, x, c=spl, s=1, cmap=plt.cm.bone, norm=colors.SymLogNorm(linthresh=10000, linscale=10000, vmin=min_splrms, vmax=max_splrms))
fig.colorbar(spl_plot, ax=ax0)
ax0.set_title("Sound pressure@Int-12. RMS values [Pa]")
dbl_plot = ax1.scatter(z, x, c=dbl, s=1, vmin=min_dblrms, vmax=max_dblrms, cmap=plt.cm.bone)
fig.colorbar(dbl_plot, ax=ax1)
ax1.set_title("dB level@Int-12. RMS values [dB]")
#plt.show()
os.chdir(path_plots)
plt.savefig(str('int-12_acu_RMS.png'))
print("Plotting RMS done...")

Plotting RMS values...
Plotting RMS done...


In [ ]:
print("Starting plotting loop...")
os.chdir(path_acu)
filelist = os.listdir(path_acu)
for file in filelist:
    timestep = str(os.path.basename(str(file)))[11:-4]
    os.chdir(path_acu)
    acu = pd.read_csv(file, sep=',', decimal='.', header=0)
    x = acu['x-coordinate']
    y = acu['y-coordinate']
    z = acu['z-coordinate']
    spl = acu['sound-pressure']
    dbl = acu['db-level']
    fig, (ax0, ax1) = plt.subplots(nrows=2, figsize=(10, 10), dpi=300)
    spl_plot = ax0.scatter(z, x, c=spl, s=2, cmap=plt.cm.bone, norm=colors.SymLogNorm(linthresh=5000, linscale=5000, vmin=-15000, vmax=12000))
    fig.colorbar(spl_plot, ax=ax0)
    ax0.set_title(str('Sound pressure. Int-01. Time: ' + str(timestep)))
    dbl_plot = ax1.scatter(z, x, c=dbl, s=2, vmin=min_dbl, vmax=max_dbl, cmap=plt.cm.bone)
    fig.colorbar(dbl_plot, ax=ax1)
    ax1.set_title(str('dB level. Int-01. Time: ' + str(timestep)))
    os.chdir(path_plots)
    plt.savefig(str('int-01_acu_t_' + str(timestep) + '.png'))
    plt.close()
    print(str('int-01_acu_t_' + str(timestep) + '.png done...'))
print("Exiting plotting loop...")